In [30]:
import pandas as pd
from math import sin,cos
from mft1 import find_max_abs
import mft1_2
h = 0.1 #Шаг интерполяции

# Начальные данные
### Функция $f(x)=\cos(x), [-0.5;0.5]$

In [31]:
pivot = 0.05
a = -0.5
b = 0.5
f = lambda x: cos(x)
d1f = lambda x: -sin(x)
d2f = lambda x: -cos(x)
d3f = lambda x: sin(x)
d4f = lambda x: cos(x)
d5f = lambda x: -sin(x)

In [32]:
def get_values(f,a,b,h):
    n = int((b-a)/h)+1
    return [f(a+k*h) for k in range(n)]
    
def buildTable1(f,a,b,h): 
    n = int((b-a)/h)+1
    nodes = [(a+k*h) for k in range(n)]
    values = get_values(f,a,b,h)
    return pd.DataFrame([nodes,values],index = ["x","f(x)"], columns = [" "]*n).T

df = buildTable1(f,a,b,h)
df

,x,f(x)
,-0.5,0.877583
,-0.4,0.921061
,-0.3,0.955336
,-0.2,0.980067
,-0.1,0.995004
,0.0,1.000000
,0.1,0.995004
,0.2,0.980067
,0.3,0.955336
,0.4,0.921061


In [33]:
def get_ERs(f, a, b):
    values = get_values(f,a,b,h)
    le = len(values)
    ER = [[]]*4
    ER[0] = [values[k+1]-values[k] for k in range(le-1)]
    for i in range(1,4):
        ER[i] = [ER[i-1][k+1]-ER[i-1][k] for k in range(le-1-i)]
    return ER
    
ER = get_ERs(f, a ,b)
ER

[[0.043478432112512344,
  0.03427549512272088,
  0.024730088715635645,
  0.014937587436784194,
  0.0049958347219741794,
  -0.0049958347219742905,
  -0.014937587436784083,
  -0.024730088715635645,
  -0.03427549512272088,
  -0.043478432112512344],
 [-0.009202936989791466,
  -0.009545406407085233,
  -0.009792501278851451,
  -0.009941752714810015,
  -0.00999166944394847,
  -0.009941752714809793,
  -0.009792501278851562,
  -0.009545406407085233,
  -0.009202936989791466],
 [-0.000342469417293767,
  -0.00024709487176621803,
  -0.00014925143595856394,
  -4.991672913845502e-05,
  4.991672913867706e-05,
  0.00014925143595823087,
  0.00024709487176632905,
  0.000342469417293767],
 [9.537454552754898e-05,
  9.784343580765409e-05,
  9.933470682010892e-05,
  9.983345827713208e-05,
  9.933470681955381e-05,
  9.784343580809818e-05,
  9.537454552743796e-05]]

In [34]:
def buildTable2(f,a,b,h):
    n = int((b-a)/h)+1
    nodes = [a+(k//2)*h if k % 2 ==0 else ' ' for k in range(2*n-1)]
    values = [f(a+(k//2)*h) if k % 2 ==0 else ' ' for k in range(2*n-1)]
    ER_buf = get_ERs(f,a,b)
    for i in range(4):
        ER[i] = [' ']*i + [round(ER_buf[i][k//2],5) if k % 2 == 1 else ' ' for k in range(2*(n-i)-1)] + [' ']*i
    return pd.DataFrame([nodes,values,ER[0],ER[1],ER[2],ER[3]], index = ["x","y",'$ \Delta y $','$ \Delta^{2} y $'
                                    ,'$ \Delta^{3} y $','$ \Delta^{4} y $']).T
# Написать единую функцию для конечных разностей (они имеют капец какое одинаковое построение)
df1 = buildTable2(f,a,b,h)
df1

,x,y,$ \Delta y $,$ \Delta^{2} y $,$ \Delta^{3} y $,$ \Delta^{4} y $
0,-0.5,0.877583,,,,
1,,,0.04348,,,
2,-0.4,0.921061,,-0.0092,,
3,,,0.03428,,-0.00034,
4,-0.3,0.955336,,-0.00955,,0.0001
5,,,0.02473,,-0.00025,
6,-0.2,0.980067,,-0.00979,,0.0001
7,,,0.01494,,-0.00015,
8,-0.1,0.995004,,-0.00994,,0.0001
9,,,0.005,,-5e-05,


In [35]:
def calculate_Ns(a,b,h,pivot):
    N = [1]+[0]*5
    if pivot <= a + h/2:
        t = (pivot-a)/h
        for k in range(1,6):
            N[k] = N[k-1]*(t-k+1)/k
    elif pivot >= b - h/2:
        t = (pivot-b)/h
        for k in range(1,6):
            N[k] = N[k-1]*(t+k-1)/k
    else:
        mid = a+h*int((b-a)/h)/2
        if mid < pivot <= mid+h/2:
            t = (pivot-mid)/h
            for k in range(1,6):
                N[k] = N[k-1]*(t-(-1)**k * k//2)/k
    return N

N = calculate_Ns(a,b,h,pivot)
N

[1, 1.5, -0.375, -0.3125, 0.1171875, 0.08203125]

In [36]:
def calculate_Pks(a,b,h,values,N,ER):
    Pks = [0]*5
    ER0 = [0]*5
    if pivot <= a + h/2:
        ER0 = [ER[k][0] for k in range(4)]
        Pks[0] = values[0]
    else: 
        n = len(values)
        if pivot >= b - h/2:
            print(n)
            ER0 = [ER[k][n-2-k] for k in range(4)]
            Pks[0] = values[n-1]
        else:
            mid = a+h*int((b-a)/h)/2
            if mid < pivot <= mid+h/2:
                ER0 = [ER[(n-1-k)//2] for k in range(4)]
                Pks[0] = values[n//2]
    for k in range(0,4):
        Pks[k+1] = Pks[k] + ER0[k]*N[k+1]
    return Pks, [values[0]] + ER0

P, ER0 = calculate_Pks(a,b,h,get_values(f,a,b,h),N,ER)
P
    

IndexError: list index out of range

In [ ]:
def buildTable3(f,derivatives,pivot,a,b,h,ER, N, P):  # Строит таблицу 1 по образцу
    string1 = ER

    string2 = N[:-1]

    string3 = [ER[k]*N[k] for k in range(5)]

    string4 = P

    string5 = [f(pivot)-P[k] for k in range(5)]
    
    maximums = [0]*5
    if pivot <= a + h/2:
        maximums[0] = abs(d1f(a))
        for k in range(1,5):
            maximums[k] = find_max_abs(derivatives[k],[a,a+k*h])
        
    else: 
        if pivot >= b-h/2:
            maximums[0] = abs(d1f(b))
            for k in range(1,5):
                maximums[k] = find_max_abs(derivatives[k],[b-k*h,b])
        
        else:   
            mid = a+h*int((b-a)/h)/2 
            if mid < pivot <= mid+h/2:
                maximums[0] = abs(d1f(mid))
                for k in range(1,5):
                    maximums[k] = find_max_abs(derivatives[k],
                                               [mid-(k//2)*h,mid+((k+1)//2)*h])
    string6 = [0]*5 
    buf = h
    for k in range(0,5):
        string6[k] = maximums[k]*abs(N[k+1])*buf
        buf*=h

    return pd.DataFrame([string1, string2, string3, string4, string5, string6],
                        index=['$ \Delta^{k} y_{0} $', "$N_{k}(t)$",
                               "$ N_{k}(t) \cdot \Delta^{k} y_{0} $",
                               "$P_{k}($"+str(pivot)+"$)$",
                               "$f($"+str(pivot)+"$) {-} P_{k}($"+str(pivot)+"$)$",
                               "$|R_{k}($"+str(pivot)+"$)|\le$"], columns=[str(k) for k in range(5)])

df2 = buildTable3(f,[d1f,d2f,d3f,d4f,d5f],pivot,a,b,h,ER0,N,P)
df2
    